# README
##1. Ensure that the fields to-be-assessed in your Input DataFrame HAVE THE SAME COLUMN NAMES as the "data-quality-rules-library.json" file - ["fields"]
##2. Do remove/add the fields you want to assess under "data-quality-rules-library.json" file - ["fields"]

##4. (optional) you can add more validity rules in "data-quality-rules-library.json" file under ["rules] following the same pattern

#0. Load the Dataframe to be assessed [ TO UPDATE WITH YOUR DATAFRAME]
## By default, a Random Dataframe is generate to compute the DQ 


In [0]:
import numpy as np
from datetime import datetime
import pandas as pd

# Function to generate a random email address
def random_email():
    domains = ["example.com", "test.com", "demo.com"]
    prefix = "".join(np.random.choice(list("abcdefghijklmnopqrstuvwxyz"), size=np.random.randint(5, 10)))
    return f"{prefix}@{np.random.choice(domains)}"

# Function to generate random data for df_policies
def generate_random_policies(num_records):
    policy_data = {
        "Policy_ID": [f"POL{np.random.randint(1000, 9999)}" for _ in range(num_records)],
        "Holder_Name": [f"Holder_{np.random.randint(1, 99)}" for _ in range(num_records)],
        "Holder_Age": [np.random.choice([np.nan, np.random.randint(16, 100), "null"], p=[0.1, 0.85, 0.05]) for _ in range(num_records)],
        "Holder_Gender": [np.random.choice(["Male", "Female", "", "nil"], p=[0.49, 0.49, 0.01, 0.01]) for _ in range(num_records)],
        "Vehicle_Type": [np.random.choice(["Sedan", "Coupe", "Hatchback", "Pickup", "SUV", "null"], p=[0.2, 0.2, 0.2, 0.2, 0.19, 0.01]) for _ in range(num_records)],
        "Vehicle_Age": [np.random.choice([np.nan, np.random.randint(1, 20)], p=[0.1, 0.9]) for _ in range(num_records)],
        "Vehicle_Value": [np.random.randint(5000, 30000) if np.random.rand() > 0.1 else "n/a" for _ in range(num_records)],
        "Policy_Start_Date": [datetime.strftime(datetime.now() + pd.Timedelta(days=np.random.randint(1, 365)), "%Y-%m-%d") if np.random.rand() > 0.1 else "" for _ in range(num_records)],
        "Policy_End_Date": [datetime.strftime(datetime.now() + pd.Timedelta(days=np.random.randint(365, 730)), "%Y-%m-%d") if np.random.rand() > 0.1 else " " for _ in range(num_records)],
        "Premium_Amount": [np.random.randint(200, 1000) if np.random.rand() > 0.1 else "--" for _ in range(num_records)],
        "Coverage_Type": [np.random.choice(["Liability Only", "Full Coverage"], p=[0.5, 0.5]) if np.random.rand() > 0.1 else "-" for _ in range(num_records)],
        "Deductible": [np.random.choice([500, 1000, 1500], p=[0.33, 0.33, 0.34]) if np.random.rand() > 0.1 else np.nan for _ in range(num_records)],
        "Claim_History": [np.random.randint(0, 5) if np.random.rand() > 0.1 else "na" for _ in range(num_records)],
        "Policy_Status": [np.random.choice(["Active", "Lapsed"], p=[0.7, 0.3]) if np.random.rand() > 0.1 else "" for _ in range(num_records)],
        "Email_Address": [random_email() if np.random.rand() > 0.1 else "invalidemail@" for _ in range(num_records)],
        "Phone_Number": [f"+1-555-{np.random.randint(1000, 9999)}-{np.random.randint(1000, 9999)}" if np.random.rand() > 0.1 else "000-0000" for _ in range(num_records)]
    }

    return pd.DataFrame(policy_data)

# Generate a random df_policies dataframe with 10 records for demonstration
random_df_policies = generate_random_policies(10)
random_df_policies


,Policy_ID,Holder_Name,Holder_Age,Holder_Gender,Vehicle_Type,Vehicle_Age,Vehicle_Value,Policy_Start_Date,Policy_End_Date,Premium_Amount,Coverage_Type,Deductible,Claim_History,Policy_Status,Email_Address,Phone_Number
0,POL5745,Holder_94,null,Female,Sedan,18.0,24560,2024-08-23,,581,Full Coverage,1000.0,0,Active,szifoeuu@demo.com,000-0000
1,POL1687,Holder_26,16,Female,Hatchback,9.0,20942,2024-10-26,2025-02-19,996,Full Coverage,1000.0,4,Active,awddmphhd@example.com,+1-555-5924-9637
2,POL1178,Holder_38,34,Male,Pickup,3.0,23594,2024-08-06,2025-07-13,883,Full Coverage,1500.0,3,Lapsed,evnuph@example.com,+1-555-4321-9700
3,POL7921,Holder_56,48,Male,Sedan,17.0,13882,2024-10-09,,599,-,NaN,4,,fbuvto@test.com,+1-555-1581-7712
4,POL8697,Holder_55,nan,Male,Sedan,10.0,5450,2023-11-07,,842,Full Coverage,500.0,0,,rjenbzpsu@test.com,+1-555-3143-7320
5,POL6870,Holder_49,28,Male,Coupe,15.0,6143,2024-02-18,2025-03-06,847,Full Coverage,500.0,0,Active,apohhrgtf@test.com,+1-555-5914-8344
6,POL1920,Holder_71,35,Female,Hatchback,17.0,n/a,2023-12-22,2025-03-25,398,Full Coverage,NaN,3,Active,fykkpyqd@example.com,+1-555-9880-5535
7,POL8101,Holder_45,81,Male,Hatchback,1.0,20251,2024-01-04,2025-04-08,979,Liability Only,500.0,3,,polnw@example.com,+1-555-4096-7360
8,POL1797,Holder_8,83,Male,Pickup,19.0,n/a,2023-11-09,2025-09-30,594,-,1500.0,na,Active,erkavzevo@example.com,+1-555-6491-2794
9,POL4787,Holder_58,44,Male,Hatchback,1.0,29529,2024-04-24,2025-10-28,--,Liability Only,500.0,2,Lapsed,jdyloqinx@demo.com,+1-555-6533-4434


#1.Load Business Rules Conf file (NO NEED TO UPDATE)

In [0]:
print("load business rules conf")
def load_business_rules(file_path):
    # Open the file and load the JSON content into a dictionary
    with open(file_path, 'r') as file:
        rules = json.load(file)
    return rules

# Example usage:
business_rules_path = '/Workspace/20231103.ADA.DQ/data-quality-rules-library.json'
config_json = load_business_rules(business_rules_path)


load business rules conf


# 
# /!\ /!\ /!\ /!\ NO UPDATES BEYOND THIS POINT  /!\ /!\ /!\ /!\
# 

#2.Run DQ Assessment

In [0]:
import pandas as pd
import json
from datetime import datetime
import re


df_policies = random_df_policies

# Step 2: Create a Sample Configuration JSON Object
# This is a simplified version of the rules that would be in the config file.


def load_business_rules(file_path):
    # Open the file and load the JSON content into a dictionary
    with open(file_path, 'r') as file:
        rules = json.load(file)
    return rules


business_rules_path = '/Workspace/20231103.ADA.DQ/data-quality-rules-library.json'
config_json = load_business_rules(business_rules_path)

df_policies, config_json

# Define the functions for DQ checks

def check_completeness(value):
    # Check if the value is a string and if it's empty or represents a missing value
    if isinstance(value, str):
        value = value.strip().lower()  # Trim whitespace and convert to lowercase
        if value in ["", "null", "nil", "n/a", "na", "-", "--"]:
            return False
    return not pd.isnull(value)


def check_validity(field, value, rules):
    if pd.isnull(value) or value in ["", "null", "nan", "N/A", "n/a"]:
        return False  # Incomplete data is also invalid
    if "type" in rules:
        if rules["type"] == "integer":
            try:
                int_value = int(value)  # Attempt to convert to integer
            except ValueError:
                return False  # Non-integer value
            if "min" in rules and int_value < rules["min"]:
                return False
            if "max" in rules and int_value > rules["max"]:
                return False
    if "regex" in rules:
        if not re.match(rules["regex"], str(value)):
            return False
    if "date_format" in rules:
        try:
            datetime.strptime(str(value), rules["date_format"])
        except ValueError:
            return False
    return True


def check_accuracy(field, value, rules):
    # Placeholder for a custom function based on rules
    # Assuming a function for accuracy check is defined elsewhere and can be called here.
    # For now, we just return True, as we don't have any specific logic to check accuracy.
    return True

# Update the calculation of the Data Quality metrics to use the correct formula

def calculate_data_quality_metrics(df, dq_flags):
    # Calculate DQ metrics for each field
    dq_metrics = {}
    for field, (comp, valid, acc) in dq_flags.items():
        completeness = df[comp].mean()
        validity = df[df[comp]][valid].mean()  # % Valid among complete
        accuracy = df[df[valid]][acc].mean()  # % Accurate among valid
        data_quality = completeness * validity * accuracy
        dq_metrics[field] = {
            "Completeness": completeness,
            "Validity": validity,
            "Accuracy": accuracy,
            "Data Quality": data_quality
        }

    # Calculate overall DQ metrics
    overall_completeness = df[[comp for comp, _, _ in dq_flags.values()]].mean().mean()
    overall_validity = df[[valid for _, valid, _ in dq_flags.values()]].mean().mean()
    overall_accuracy = df[[acc for _, _, acc in dq_flags.values()]].mean().mean()
    overall_dq = overall_completeness * overall_validity * overall_accuracy

    dq_metrics["Overall"] = {
        "Completeness": overall_completeness,
        "Validity": overall_validity,
        "Accuracy": overall_accuracy,
        "Data Quality": overall_dq
    }

    return dq_metrics

# Update the main function to include the correct DQ metric calculation
def calculate_data_quality(df, config):
    dq_flags = {}

    for field in config['fields']:
        completeness_col = f"{field}__Completeness"
        validity_col = f"{field}__Validity"
        accuracy_col = f"{field}__Accuracy"

        df[completeness_col] = df[field].apply(check_completeness)
        df[validity_col] = df.apply(lambda row: check_validity(field, row[field], config['rules'].get(field, {}).get('validity', {})), axis=1)
        df[accuracy_col] = df.apply(lambda row: check_accuracy(field, row[field], config['rules'].get(field, {}).get('accuracy', {})), axis=1)

        dq_flags[field] = (completeness_col, validity_col, accuracy_col)

    dq_metrics = calculate_data_quality_metrics(df, dq_flags)

    # Create a summary dataframe for the DQ metrics
    dq_summary = pd.DataFrame.from_dict(dq_metrics, orient='index')

    return df, dq_summary

# Apply the updated completeness check and DQ metric calculation to the dataframe
df_processed, dq_summary = calculate_data_quality(df_policies, config)
dq_summary



,Completeness,Validity,Accuracy,Data Quality
Policy_ID,1.000,1.000000,1.0,1.000000
Holder_Age,0.900,0.777778,1.0,0.700000
Email_Address,1.000,1.000000,1.0,1.000000
Policy_Start_Date,1.000,1.000000,1.0,1.000000
Overall,0.975,0.925000,1.0,0.901875


In [0]:
df_processed[df_processed.filter(regex='(__Validity|__Completeness|__Accuracy)$').columns]

,Policy_ID__Completeness,Policy_ID__Validity,Policy_ID__Accuracy,Holder_Age__Completeness,Holder_Age__Validity,Holder_Age__Accuracy,Email_Address__Completeness,Email_Address__Validity,Email_Address__Accuracy,Policy_Start_Date__Completeness,Policy_Start_Date__Validity,Policy_Start_Date__Accuracy
0,True,True,True,False,False,True,True,True,True,True,True,True
1,True,True,True,True,False,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,False,True,True,True,True,True,True,True
5,True,True,True,True,True,True,True,True,True,True,True,True
6,True,True,True,True,True,True,True,True,True,True,True,True
7,True,True,True,True,True,True,True,True,True,True,True,True
8,True,True,True,True,True,True,True,True,True,True,True,True
9,True,True,True,True,True,True,True,True,True,True,True,True


In [0]:
dq_summary

,Completeness,Validity,Accuracy,Data Quality
Policy_ID,1.000,1.000000,1.0,1.000000
Holder_Age,0.900,0.777778,1.0,0.700000
Email_Address,1.000,1.000000,1.0,1.000000
Policy_Start_Date,1.000,1.000000,1.0,1.000000
Overall,0.975,0.925000,1.0,0.901875
